In [ ]:
# Import relevant modules
import matplotlib
import matplotlib.pyplot as plt
import numpy as np
from classy import Class
import pybird
from importlib import reload

In [ ]:
# Plotting style
font = {'size': 16, 'family': 'STIXGeneral'}
axislabelfontsize='large'
matplotlib.rc('font', **font)
matplotlib.mathtext.rcParams['legend.fontsize']='medium'
plt.rcParams["figure.figsize"] = [8.0,6.0]

In [ ]:
# Load Greens Functions

#GF = GreenFunction(self.Omega0_m, w=self.w0, quintessence=self.co.quintessence, MG=self.co.MG, Omega_rc=self.co.Omega_rc, nDGP=self.co.nDGP)
GF_LCDM = pybird.GreenFunction(0.25, -1., False, False, None, False)
GF_wCDM = pybird.GreenFunction(0.25, -0.9, False, False, None, False)
GF_quint = pybird.GreenFunction(0.25, -0.9, True, False, None, False)
GF_nDGP = pybird.GreenFunction(0.25, -1., False, True, 0.5, True)

In [ ]:
# Compute growth and decay
a_arr = np.logspace(-3, 0., 100)
LCDM_D_arr = np.zeros((len(a_arr),))
wCDM_D_arr = np.zeros((len(a_arr),))
quint_D_arr = np.zeros((len(a_arr),))
nDGP_D_arr = np.zeros((len(a_arr),))
LCDM_Dminus_arr = np.zeros((len(a_arr),))
wCDM_Dminus_arr = np.zeros((len(a_arr),))
quint_Dminus_arr = np.zeros((len(a_arr),))
nDGP_Dminus_arr = np.zeros((len(a_arr),))
for ia, av in enumerate(a_arr):
    LCDM_D_arr[ia] = GF_LCDM.D(av)
    wCDM_D_arr[ia] = GF_wCDM.D(av)
    quint_D_arr[ia] = GF_quint.D(av)
    nDGP_D_arr[ia] = GF_nDGP.D(av)
    LCDM_Dminus_arr[ia] = GF_LCDM.Dminus(av)
    wCDM_Dminus_arr[ia] = GF_wCDM.Dminus(av)
    quint_Dminus_arr[ia] = GF_quint.Dminus(av)
    nDGP_Dminus_arr[ia] = GF_nDGP.Dminus(av)

In [ ]:
# Take ratios to LCDM
wCDM_LCDM_D_ratio_arr = [a/b for a, b in zip(wCDM_D_arr, LCDM_D_arr)]
quint_LCDM_D_ratio_arr = [a/b for a, b in zip(quint_D_arr, LCDM_D_arr)]
nDGP_LCDM_D_ratio_arr = [a/b for a, b in zip(nDGP_D_arr, LCDM_D_arr)]
wCDM_LCDM_Dminus_ratio_arr = [a/b for a, b in zip(wCDM_Dminus_arr, LCDM_Dminus_arr)]
quint_LCDM_Dminus_ratio_arr = [a/b for a, b in zip(quint_Dminus_arr, LCDM_Dminus_arr)]
nDGP_LCDM_Dminus_ratio_arr = [a/b for a, b in zip(nDGP_Dminus_arr, LCDM_Dminus_arr)]

In [ ]:
# Plot growth
plt.figure()
plt.semilogx(a_arr, np.ones((len(a_arr),)), 'k-')
plt.semilogx(a_arr, wCDM_LCDM_D_ratio_arr, 'm-.', label='wCDM w=-0.9')
plt.semilogx(a_arr, quint_LCDM_D_ratio_arr, 'b--', label='Quintessence w=-0.9')
plt.semilogx(a_arr, nDGP_LCDM_D_ratio_arr, 'g:', label='nDGP Omega_rc=0.5')
plt.xlabel(r'$a$')
plt.ylabel(r'$D(a)$')
plt.legend()
plt.show()

In [ ]:
# Plot decay
plt.figure()
plt.semilogx(a_arr, np.ones((len(a_arr),)), 'k-')
plt.semilogx(a_arr, wCDM_LCDM_Dminus_ratio_arr, 'm-.', label='wCDM w=-0.9')
plt.semilogx(a_arr, quint_LCDM_Dminus_ratio_arr, 'b--', label='Quintessence w=-0.9')
plt.semilogx(a_arr, nDGP_LCDM_Dminus_ratio_arr, 'g:', label='nDGP Omega_rc=0.5')
plt.xlabel(r'$a$')
plt.ylabel(r'$D_{\rm minus}(a)$')
plt.legend()
plt.show()

In [ ]:
#
# As a result of the crazy result for the nDGP decay mode
# all the Greens function integrals are very slow
#

In [ ]:
# Try to get real-space matter power-spectrum for nDGP
zpk = 1.
kdata = np.linspace(0.005, 0.25, 50)
cosmo_nDGP = {'ln10^{10}A_s': 3.044,
       'n_s': 0.9649,
       'h': 0.6736,
       'omega_b': 0.02237,
       'omega_cdm': 0.120,
      }
bs = [2., 0.8, 0.2, 0.8, 0.2, -4., 0]
bdict_mmult0 = {"cct": bs[4]}

reload(pybird)
correlator_nDGP = pybird.Correlator()
correlator_nDGP.set({
    'output': 'mPk', #mPk, bPk, mCf, bCf, bmPk, bmCf, w (angular Cf)
    'multipole': 0, #0 =real, 2=mono+quad, 3=mono+quad+hexadec
    'with_exact_time': True,
    'xdata': kdata,
    'z': zpk,
    'optiresum': False,
    'kmax': 0.3,
    'with_AP': False,
    'with_nDGP': True,
    #'with_bias': True,
    'Omega_rc': 0.5,
})
correlator_nDGP.compute(cosmo_nDGP, module='class')
mPk_nDGP = correlator_nDGP.get(bdict_mmult0)

In [ ]:
print(mPk_nDGP)